In [44]:
import easyocr
import cv2
import matplotlib.pyplot as plt
from icecream import ic
import os

In [45]:
def get_ocr_results(image_path):
    reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
    result = reader.readtext(image_path)
    return result

In [46]:
def get_coordinates_for_text(data, target_text):
    matching_coordinates = []

    for item in data:
        coordinates, ocr_text, _ = item  # We ignore the confidence score here
        if target_text.lower() in ocr_text.lower() :  # Case-insensitive comparison
            matching_coordinates.append(coordinates)

    return matching_coordinates

In [91]:
import cv2
import os

def draw_rectangle_on_video(input_video_path, output_video_path, top_left_coordinate, bottom_right_coordinate,band_color):
    """
    Draws a solid rectangle on a video with error handling.

    Parameters:
    - input_video_path: Path to the input video file.
    - output_video_path: Path where the output video will be saved.
    - top_left_coordinate: Tuple (x, y) for the top left coordinate of the rectangle.
    - bottom_right_coordinate: Tuple (x, y) for the bottom right coordinate of the rectangle.
    """
    try:
        # Attempt to open the input video
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            raise IOError(f"Could not open video file: {input_video_path}")

        # Get the video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Generate output video path by appending '_output' before the file extension
        # base_name= os.path.basename(input_video_path)
        output_video_path = f"{output_video_path}.mp4"
        ic(output_video_path)

        # Define the codec and create a VideoWriter object
        out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

        frame_count = 0  # To count the number of frames processed

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Draw a solid rectangle on the frame
            cv2.rectangle(frame, top_left_coordinate, bottom_right_coordinate, band_color, -1)

            # Write the frame into the file
            out.write(frame)
            frame_count += 1

        print(f"Processed {frame_count} frames.")
    except IOError as e:
        print(f"IOError: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Release resources in the finally block to ensure they are always executed
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        if 'out' in locals():
            out.release()

In [79]:
def get_mid_frame(video_path,output_path = '',save_path=False):
    # Read the Video (Change the filename as per your file)
    cap = cv2.VideoCapture(video_path)

    # Get the total number of frames
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    half_point = length//2 # Approximately half if number of frames are odd

    # Set the reader to the given frame number (half_point)
    cap.set(cv2.CAP_PROP_POS_FRAMES, half_point)

    # Read the frame
    ret, frame = cap.read()

    base_name = os.path.basename(video_path)
    print(base_name)
    if save_path: cv2.imwrite(f"{output_path}{base_name}_mid.png", frame)

    # Release the file pointer
    cap.release()
    return frame
# mid_frame = get_mid_frame(os.path.join('downloads','10_Parents Built a Beautiful Bedroom For Son & Daughter  3d Animation By The Stories Time  shorts.mp4'),output_path='',save_path=True)
# mid_frame = get_mid_frame('test.mp4',output_path='mid_frames/',save_path=True)
# mid_frame

### Actual code

In [80]:
mid_frame = get_mid_frame(r'downloads/10_Parents Built a Beautiful Bedroom For Son & Daughter  3d Animation By The Stories Time  shorts.mp4',output_path='mid_frames/',save_path=True)

10_Parents Built a Beautiful Bedroom For Son & Daughter  3d Animation By The Stories Time  shorts.mp4


In [60]:
ocr_results = get_ocr_results(mid_frame)
ocr_results

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[61, 55], [109, 55], [109, 69], [61, 69]], 'BEST', 0.9950381817750114),
 ([[28, 64], [136, 64], [136, 88], [28, 88]], 'INDIAN', 0.9313570263730901),
 ([[46, 86], [78, 86], [78, 94], [46, 94]], 'ILNE', 0.12265399098396301),
 ([[81, 83], [135, 83], [135, 97], [81, 97]], 'QAMINO', 0.46622896014818577),
 ([[47, 95], [119, 95], [119, 107], [47, 107]],
  'PLAIDEM',
  0.09117240543804958),
 ([[262, 1058], [462, 1058], [462, 1088], [262, 1088]],
  'The Stories Time',
  0.9996257111612783)]

In [61]:
matches = get_coordinates_for_text(ocr_results,'stories')
matches

[[[262, 1058], [462, 1058], [462, 1088], [262, 1088]]]

In [52]:
def get_mid_band_color(image,top_left):
    color = (image[top_left[1],top_left[0]])
    blue,green,red = int(color[0]),int(color[1]),int(color[2])
    ic(color)
    return color

band_color = get_mid_band_color(mid_frame,[262, 1058])

ic| color: array([ 70,  80, 118], dtype=uint8)


In [53]:
top_left = matches[0][0]
ic(top_left)
bottom_right = matches[0][2]

top_left[1]-=15
bottom_right[1]+=15
ic(top_left)

ic| top_left: [262, 1058]
ic| top_left:

 [262, 1043]


[262, 1043]

In [95]:
draw_rectangle_on_video('test.mp4','stage1/test_output',top_left_coordinate=top_left,bottom_right_coordinate=bottom_right,band_color = [ 70,  80, 118])

ic| output_video_path: 'stage1/test_output.mp4'


Processed 1318 frames.


## Loop

In [55]:
filenames = next(os.walk('downloads'), (None, None, []))[2]  

In [99]:
filenames_s = sorted(filenames, key=lambda x: int(x.split('_')[0]))

In [101]:
for raw_name in list(reversed(filenames_s))[:1]:
    v_name = raw_name.split('3d')[0].strip()+' 3D home redesign animations'+'_stage1'
    filepath = os.path.join('downloads',raw_name)
    ic(filepath)
    ic(v_name)
    mid_frame = get_mid_frame(filepath,output_path='mid_frames/',save_path=True)
    ocr_results = get_ocr_results(mid_frame)
    matches = get_coordinates_for_text(ocr_results,'stories')
    band_color = get_mid_band_color(mid_frame,[262, 1058])
    top_left = matches[0][0]
    bottom_right = matches[0][2]
    top_left[1]-=15
    bottom_right[1]+=15
    draw_rectangle_on_video(filepath,'stage1/'+v_name,top_left_coordinate=top_left,bottom_right_coordinate=bottom_right,band_color = [ 70,  80, 118])

ic| filepath: 'downloads\\74_Turn a Simple Room Into a Luxury Room in 3d Animation shorts.mp4'
ic| v_name

: '74_Turn a Simple Room Into a Luxury Room in 3D home redesign animations_stage1'
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


74_Turn a Simple Room Into a Luxury Room in 3d Animation shorts.mp4


ic| color: array([137, 165, 190], dtype=uint8)
ic| output_video_path: ('stage1/74_Turn a Simple Room Into a Luxury Room in 3D home redesign '
                        'animations_stage1.mp4')


Processed 1690 frames.
